In [1]:
from stanfordcorenlp import StanfordCoreNLP
from stanfordcorenlp import StanfordCoreNLP as stnlp
nlp = stnlp(r'/home/ayush/stanford-corenlp-full-2018-01-31')

In [2]:
def getMapping(sentence):
    open_bracket = []
    mapping = {}
    for i,j in enumerate(sentence):
        if(j=='('):
            open_bracket.append(i)
        elif (j==')'):
            mapping[open_bracket[-1]] = i
            del open_bracket[-1]
    return mapping

In [99]:
#OUTPUT in form of [NP[VP[NP[VP[NP...]]]]]
def SVO(sentence):
    sentence = nlp.parse(sentence)
    sentence = str(sentence)
    ar = [a.strip() for a in sentence.split('\n')]
    sentence = ''.join(ar)
    return getTags(sentence)

def getTags(sentence):
    mapping = getMapping(sentence)
    return getNP(sentence, 0, mapping)
        
def getNP(sentence, idxStart, mapping):
    res = []
    idxNP = sentence.index('(NP')
    idxNPend = mapping[idxStart + idxNP] - idxStart
    subj = sentence[idxNP:idxNPend]
    if '(VP' not in sentence[idxNP:idxNPend]:
        remain = sentence[idxNPend+1:]
        if '(VP' in remain:
            verblist = getVP(remain, idxNPend+1+idxStart, mapping)
            res.append(subj)
            res.append(verblist)
        else:
            res.append(subj)
    else:
        #NP(VP-NP)
        #NP(VP(NP))
        remain = sentence[idxNP:idxNPend]
        verblist = getVP(remain, idxNP+idxStart, mapping)
        if type(verblist) == type('str'):
            temp = verblist
        else:
            temp = verblist[0]
        while (type(temp)!=type('str')):
            temp = temp[0]
        if temp in subj:
            idxx = subj.index(temp)
            subj = subj[:idxx]
        res.append(subj)
        res.append(verblist)
    
    return res
        
def getVP(sentence, idxStart, mapping):
    res = []
    idxVP = sentence.index('(VP')
    idxVPend = mapping[idxStart + idxVP] - idxStart
    verb = sentence[idxVP:idxVPend]
    if '(NP' not in sentence[idxVP:idxVPend]:
        #VP-NP
        remain = sentence[idxVPend+1:]
        if '(NP' in remain:
            nplist = getNP(remain, idxVPend+1+idxStart, mapping)
            res.append(verb)
            res.append(nplist)
        else:
            res.append(verb)
    
    else:
        remain = sentence[idxVP:idxVPend]
        nplist = getNP(remain, idxVP+idxStart, mapping)
        if type(nplist) == type('str'):
            temp = nplist
        else:
            temp = nplist[0]
        while (type(temp)!=type('str')):
            temp = temp[0]
        if temp in verb:
            idxx = verb.index(temp)
            verb = verb[:idxx]
        res.append(verb)
        res.append(nplist)
    return res

In [100]:
sen =  "Architecturally, the school has a Catholic character. Atop the Main Building\'s gold dome is a golden statue of the Virgin Mary. Immediately in front of the Main Building and facing it, is a copper statue of Christ with arms upraised with the legend \"Venite Ad Me Omnes\". Next to the Main Building is the Basilica of the Sacred Heart. Immediately behind the basilica is the Grotto, a Marian place of prayer and reflection. It is a replica of the grotto at Lourdes, France where the Virgin Mary reputedly appeared to Saint Bernadette Soubirous in 1858. At the end of the main drive (and in a direct line that connects through 3 statues and the Gold Dome), is a simple, modern stone statue of Mary."
sen = sen.split('.')
del sen[-1]
sen

['Architecturally, the school has a Catholic character',
 " Atop the Main Building's gold dome is a golden statue of the Virgin Mary",
 ' Immediately in front of the Main Building and facing it, is a copper statue of Christ with arms upraised with the legend "Venite Ad Me Omnes"',
 ' Next to the Main Building is the Basilica of the Sacred Heart',
 ' Immediately behind the basilica is the Grotto, a Marian place of prayer and reflection',
 ' It is a replica of the grotto at Lourdes, France where the Virgin Mary reputedly appeared to Saint Bernadette Soubirous in 1858',
 ' At the end of the main drive (and in a direct line that connects through 3 statues and the Gold Dome), is a simple, modern stone statue of Mary']

In [102]:
for s in sen:
    print SVO(s)
    print '----------------------------------------------------------------------------------'

['(NP (DT the) (NN school)', ['(VP (VBZ has)', ['(NP (DT a) (JJ Catholic) (NN character)']]]
----------------------------------------------------------------------------------
["(NP(NP (DT the) (NNP Main) (NNP Building) (POS 's))(NN gold)", ['(VP (VBZ is)', ['(NP(NP (DT a) (JJ golden) (NN statue))(PP (IN of)(NP (DT the) (NNP Virgin) (NNP Mary)))']]]
----------------------------------------------------------------------------------
['(NP(NP (NN front))(PP (IN of)(NP (DT the) (NNP Main) (NNP Building)))', ['(VP (VBG facing)(SBAR(S', ['(NP (PRP it)', ['(VP (VBZ is)', ['(NP(NP (DT a) (NN copper) (NN statue))(PP (IN of)(NP(NP (NNP Christ))(PP (IN with)(NP(NP (NNS arms))', ['(VP (VBN upraised)(PP (IN with)', ["(NP (DT the) (NN legend) (`` ``)(NP(NP (JJ Venite) (NN Ad))(NP(NP (PRP Me))(NNPS Omnes)))('' '')"]]]]]]]
----------------------------------------------------------------------------------
['(NP (DT the) (NNP Main) (NNP Building)', ['(VP (VBZ is)', ['(NP(NP (DT the) (NN Basilica))(PP (I

In [103]:
from nltk.stem import WordNetLemmatizer
wordnet_lemmatizer = WordNetLemmatizer()

In [111]:
for s in sen[1].split(' '):
    print s, wordnet_lemmatizer.lemmatize(s)

 
Atop Atop
the the
Main Main
Building's Building's
gold gold
dome dome
is is
a a
golden golden
statue statue
of of
the the
Virgin Virgin
Mary Mary


In [110]:
wordnet_lemmatizer.lemmatize('aardwolves')

u'aardwolf'

In [115]:
from nltk.stem.porter import PorterStemmer
porter_stemmer = PorterStemmer()

In [116]:
for s in sen[1].split(' '):
    print s, porter_stemmer.stem(s)

 
Atop atop
the the
Main main
Building's building'
gold gold
dome dome
is is
a a
golden golden
statue statu
of of
the the
Virgin virgin
Mary mari


In [117]:
sen[1]

" Atop the Main Building's gold dome is a golden statue of the Virgin Mary"